# CFiddle Tutorial and Tour

CFiddle is a Python library for studying how smallish programs are compiled and execute.  It is built to make it easy to experiment and explore.

With it you can:

1. See what the compiler did to your code.
2. Visualize programs with control flow graphs.
3. Compare the performance of different algorithms/compilation flags/inputs.
4. Easily process, plot, or export the results of the above.
5. Do all of the above in Juypter Notebook/Lab.

If you want to get a feel for why CFiddle is useful (and see some more of its features), checkout [the gallery](README.ipynb).  If you want to learn how to use CFiddle, read on!

If you have questions, contact swanson@cs.ucsd.edu.

## Importing CFiddle

Load up everything and set it up to place nicely with Jupyter.

In [ ]:
%xmode Minimal # this produces more readable exceptions.
from cfiddle import *


## Building Code

Let's compile a program:

In [ ]:
exe = build(code(r"""
extern "C"
int forty_two() {
    return 42;
}"""))[0]

There are few things going on here:

1.  `code()` takes a raw Python string, stores it in anonymous file, and returns the filename.
2.  We declared `forty_two()` as `extern "C"` so it's name won't get mangled.
2.  `build()` compiles the code in the file into a dynamic library, so we can execute it later.

`build()` returns a _list_ of objects that represent versions of the compiled code.  In this case, we only compiled one version, so the list has a single entry, which we stored in `exe`.

Before we run `forty_two()`, let's take a look at it's assembly:

In [ ]:
exe.asm("forty_two")

## Running Code

Let's run the function:

In [ ]:
results = run(exe, "forty_two")
display(results[0].return_value)

The `run()` function takes an executable and a function name to invoke.  It returns a _list_ of objects representing the results of execution. In this case, again, there's just one object, because we only executed one function, one time.

## Building Multiple Versions

Here's a more interesting function:

In [ ]:
more_interesting = code(r"""
extern "C"
int times_forty_two(int x) {
    return 42 * x;
}

extern "C"
int plus_forty_two(int x) {
    return 42 + x;
}""")

Let's compile it with a specific set of compiler flags:

In [ ]:
exe = build(source=more_interesting, 
            build_parameters=arg_map(OPTIMIZE="-O1"))

The `build_parameters` argument takes a dict that sets `Makefile` variables that CFiddle will use to build the executable, so now `exe[0]` represents a slightly more optimized version of `more_interesting`.

It'd be more interesting two compare two different optimization levels, and CFiddle makes that easy:

In [ ]:
exe = build(more_interesting, 
            build_parameters=arg_map(OPTIMIZE=["-O0", "-O1"]))

The key here is `arg_map()`.  `arg_map()` is the third central tool in CFiddle (along with `build()` and `run()`).

`arg_map()` takes a `dict` that maps argument names to lists of values.  It returns a `list` of `dict`s that map those keys to the corresponding values in all possible combinations.  For instance:

In [ ]:
arg_map(a=[1,2], b=[3,4])

Or, more relevantly:

In [ ]:
arg_map(OPTIMIZE=["-O0", "-O1"])

So the `build()` call above, passes a list of `build_parameter` settings and compiles `more_interesting` for each set of values.  The result is two executables:

In [ ]:
display(exe)

And we can compare their assembly like so:

In [ ]:
from cfiddle.jupyter import *
compare([exe[0].asm("times_forty_two"), exe[1].asm("times_forty_two")], # compare takes two things and puts them side by side.
        [html_parameters(exe[0].get_build_parameters()), html_parameters(exe[1].get_build_parameters())])  # you can provide headers, too.

## Visualizing Control Flow

CFiddle can make it much easier to read assembly language by rendering it as a control flow graph.

In [ ]:
loop = build(code(r"""
extern "C"
int loop() { 
    int sum = 0;
    for(int i = 0; i < 10; i++) {
        sum += i;
    }
    return sum;
}
"""))
compare([loop[0].asm("loop"), loop[0].cfg("loop")])

## Running Multiple Versions

We can use `arg_map()` way with `run()` as well run one or more functions with one or more sets of arguments:

In [ ]:
results = run(exe, ["times_forty_two", "plus_forty_two"], arguments=arg_map(x=[2, 4]))

That ran eight different function calls (which `as_df()` will render as a Pandas data frame):

In [ ]:
display(results.as_df())

We could also get it as an array of `dict`s:

In [ ]:
display(results.as_dicts())

or csv:

In [ ]:
results.as_csv("/tmp/out.csv")
display(open("/tmp/out.csv").read())

or json:

In [ ]:
display(results.as_json())

## Making Measurements

Let's see fast `stl::sort()` is.  First, we'll compile it with and without optimizations:

In [ ]:
sort_exe = build(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<algorithm>

extern "C"
int sort(int n) {
    std::vector<int> data(n);
    for(int i = 0;i < n; i++) {
        data[i] = i;
    }
    auto rng = fast_URBG();                       // cfiddle.hpp provides a very fast, low-quality random number generator.
    std::shuffle(data.begin(), data.end(), rng);  
    start_measurement();                          // cfiddle.hpp provides this to mark the begining of a measurement...
    std::sort(data.begin(), data.end());
    end_measurement();                            // ... and the end
    return data[0];                               // Keep the compiler from optimizing away everything.
}"""), build_parameters=arg_map(OPTIMIZE=["-O0", "-O3"]))

The header `cfiddle.hpp` provides several useful facilities:

1. A very-fast, linear-feedback-based random number generator (and necessary C++ STL boilerplate to use it)
2. `start_measurement()`/`end_measurement()` to mark regions of interest during execution.

Let's run it:

In [ ]:
sort_results = run(sort_exe, "sort", arg_map(n=[1000000])) 
sort_results.as_df()

"ET" is for "execution time".

Let's see how long it takes to set up the array, too:

In [ ]:
exe = build(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<algorithm>

extern "C"
int sort(int n) {
    start_measurement("initializing");             // The string shows up in the output in the 'tag' column      
    std::vector<int> data(n);
    for(int i = 0;i < n; i++) {
        data[i] = i;
    }
    auto rng = fast_URBG();                       
    std::shuffle(data.begin(), data.end(), rng);  
    restart_measurement("sorting");               // This ends one tag and begins another.
    std::sort(data.begin(), data.end());
    end_measurement();                           
    return data[0];                              
}"""), build_parameters=arg_map(OPTIMIZE=["-O0", "-O3"]))

In [ ]:
results = run(exe, "sort", arg_map(n=[1000000])) 
results.as_df()

## Drawing Graphs

CFiddle doesn't directly provide data visualization services, but it works well with standard, Jupyter-friendly tools like Pandas and Matplotlib:

In [ ]:
sort_results.as_df().plot.bar(x="OPTIMIZE", y="ET", ylabel="ET")

Line graphs are easy too:

In [ ]:
sort_results = run(sort_exe[1], "sort", arg_map(n=range(10000, 100000,1000))) 

In [ ]:
sort_results.as_df().plot.line(x="n", y="ET", xlabel="ET", ylabel="ET", title="OPTIMIZE = -O3")

## Limitations

There are few things to watch out for with CFiddle:

1. It uses heuristics rather than real parsing to identify function names.
    * It can't handle newlines in function prototypes.
    * It only knows about basic C/C++ types.
    * Function prototypes need to be on one line.
2. The CFG rendering use [Redare2](https://rada.re/), a very powerful reverse engineering tool.
    * It works on binaries rather than assembly (so the output of `cfg()` doesn't perfectly match the output of `asm()`)
    * Sometimes it gets confused and returns nonsense.
    * More recent versions of gcc seems to confuse it more than older versions.
3.  If your code SEGFAULTs it will take Python down with it.